In [1]:
import tensorflow as tf

# Print the TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# List all available devices
devices = tf.config.list_physical_devices("GPU")
if devices:
    print("GPU is available")
else:
    print("GPU is not available")

2024-05-30 17:20:43.345176: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


TensorFlow version: 2.4.1
GPU is available


2024-05-30 17:20:44.049470: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-05-30 17:20:44.050081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-05-30 17:20:44.185244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-30 17:20:44.189763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.59GiB deviceMemoryBandwidth: 871.81GiB/s
2024-05-30 17:20:44.189785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-05-30 17:20:44.191337: I tensorflow/stream_executor/platform/d

In [ ]:
import numpy as np 
import Pyfhel

In [ ]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()  # Creating empty Pyfhel object

ckks_params = {
    "scheme": "CKKS",
    "n": 2**14,
    "scale": 2**30,
    "qi_sizes": [60, 30, 30, 30, 60],
}

seed = 1234567890  # Set the seed value

HE.contextGen(**ckks_params)  # Generate context for CKKS scheme
HE.reseed(seed)  # Set the seed for key generation
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()  # Generate rotation keys for CKKS scheme

# Encryption
pt1 = np.array([1.5, 2.3, 3.7, 4.1])
ct1 = HE.encryptFrac(pt1)

pt2 = np.array([2.1, 3.2, 4.8, 5.6])
ct2 = HE.encryptFrac(pt2)

# Homomorphic operations
ct_add = ct1 + ct2
ct_mult = ct1 * ct2

# Decryption
pt_add = HE.decryptFrac(ct_add)
pt_mult = HE.decryptFrac(ct_mult)


In [ ]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()  # Creating empty Pyfhel object
ckks_params = {
    "scheme": "CKKS",  # can also be 'ckks'
    "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
    #  encoded in a single ciphertext.
    #  Typ. 2^D for D in [10, 15]
    "scale": 2**30,  # All the encodings will use it for float->fixed point
    #  conversion: x_fix = round(x_float * scale)
    #  You can use this as default scale or use a different
    #  scale on each operation (set in HE.encryptFrac)
    "qi_sizes": [60, 30, 30, 30, 60],  # Number of bits of each prime in the chain.
    # Intermediate values should be  close to log2(scale)
    # for each operation, to have small rounding errors.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()

In [ ]:
arr_x = np.array([0.1, 0.2, -0.3], dtype=np.float64)  # Always use type float64!
arr_y = np.array([-1.5, 2.3, 4.7], dtype=np.float64)

ptxt_x = HE.encodeFrac(arr_x)  # Creates a PyPtxt plaintext with the encoded arr_x
ptxt_y = HE.encodeFrac(
    arr_y
)  # plaintexts created from arrays shorter than 'n' are filled with zeros.

ctxt_x = HE.encryptPtxt(ptxt_x)  # Encrypts the plaintext ptxt_x and returns a PyCtxt
ctxt_y = HE.encryptPtxt(ptxt_y)  #  Alternatively you can use HE.encryptFrac(arr_y)

# Otherwise, a single call to `HE.encrypt` would detect the data type,
#  encode it and encrypt it
# > ctxt_x = HE.encrypt(arr_x)

print("\n2. Fixed-point Encoding & Encryption, ")
print("->\tarr_x ", arr_x, "\n\t==> ptxt_x ", ptxt_x, "\n\t==> ctxt_x ", ctxt_x)
print("->\tarr_y ", arr_y, "\n\t==> ptxt_y ", ptxt_y, "\n\t==> ctxt_y ", ctxt_y)

In [ ]:
arr_x = np.array([0.1, 0.2, -0.3], dtype=np.float64)  # Always use type float64!
arr_y = np.array([-1.5, 2.3, 4.7], dtype=np.float64)

ptxt_x = HE.encodeFrac(arr_x)  # Creates a PyPtxt plaintext with the encoded arr_x
ptxt_y = HE.encodeFrac(
    arr_y
)  # plaintexts created from arrays shorter than 'n' are filled with zeros.

ctxt_x = HE.encryptPtxt(ptxt_x)  # Encrypts the plaintext ptxt_x and returns a PyCtxt
ctxt_y = HE.encryptPtxt(ptxt_y)  #  Alternatively you can use HE.encryptFrac(arr_y)

# Otherwise, a single call to `HE.encrypt` would detect the data type,
#  encode it and encrypt it
# > ctxt_x = HE.encrypt(arr_x)

print("\n2. Fixed-point Encoding & Encryption, ")
print("->\tarr_x ", arr_x, "\n\t==> ptxt_x ", ptxt_x, "\n\t==> ctxt_x ", ctxt_x)
print("->\tarr_y ", arr_y, "\n\t==> ptxt_y ", ptxt_y, "\n\t==> ctxt_y ", ctxt_y)

In [ ]:
# Ciphertext-ciphertext ops:
ccSum = ctxt_x + ctxt_y  # Calls HE.add(ctxt_x, ctxt_y, in_new_ctxt=True)
#  `ctxt_x += ctxt_y` for inplace operation
ccSub = ctxt_x - ctxt_y  # Calls HE.sub(ctxt_x, ctxt_y, in_new_ctxt=True)
#  `ctxt_x -= ctxt_y` for inplace operation
ccMul = ctxt_x * ctxt_y  # Calls HE.multiply(ctxt_x, ctxt_y, in_new_ctxt=True)
#  `ctxt_x *= ctxt_y` for inplace operation
cSq = ctxt_x**2  # Calls HE.square(ctxt_x, in_new_ctxt=True)
#  `ctxt_x **= 2` for inplace operation
cNeg = -ctxt_x  # Calls HE.negate(ctxt_x, in_new_ctxt=True)
#
# cPow  = ctxt_x**3          # pow Not supported in CKKS
cRotR = ctxt_x >> 2  # Calls HE.rotate(ctxt_x, k=2, in_new_ctxt=True)
#  `ctxt_x >>= 2` for inplace operation
cRotL = ctxt_x << 2  # Calls HE.rotate(ctxt_x, k=-2, in_new_ctxt=True)
#  `ctxt_x <<= 2` for inplace operation

# Ciphetext-plaintext ops
cpSum = ctxt_x + ptxt_y  # Calls HE.add_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
# `ctxt_x += ctxt_y` for inplace operation
cpSub = ctxt_x - ptxt_y  # Calls HE.sub_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
# `ctxt_x -= ctxt_y` for inplace operation
cpMul = ctxt_x * ptxt_y  # Calls HE.multiply_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
# `ctxt_x *= ctxt_y` for inplace operation


print("3. Secure operations")
print(" Ciphertext-ciphertext: ")
print("->\tctxt_x + ctxt_y = ccSum: ", ccSum)
print("->\tctxt_x - ctxt_y = ccSub: ", ccSub)
print("->\tctxt_x * ctxt_y = ccMul: ", ccMul)
print(" Single ciphertext: ")
print("->\tctxt_x**2      = cSq  : ", cSq)
print("->\t- ctxt_x       = cNeg : ", cNeg)
print("->\tctxt_x >> 4    = cRotR: ", cRotR)
print("->\tctxt_x << 4    = cRotL: ", cRotL)
print(" Ciphertext-plaintext: ")
print("->\tctxt_x + ptxt_y = cpSum: ", cpSum)
print("->\tctxt_x - ptxt_y = cpSub: ", cpSub)
print("->\tctxt_x * ptxt_y = cpMul: ", cpMul)

In [ ]:
print("\n4. Relinearization-> Right after each multiplication.")
print(f"ccMul before relinearization (size {ccMul.size()}): {ccMul}")
HE.relinKeyGen()
~ccMul  # Equivalent to HE.relinearize(ccMul). Relin always happens in-place.
print(f"ccMul after relinearization (size {ccMul.size()}): {ccMul}")

In [ ]:
#  1. Mean
c_mean = (ctxt_x + ctxt_y) / 2
#  2. MSE
c_mse_1 = ~((ctxt_x - c_mean) ** 2)
c_mse_2 = ~((ctxt_y - c_mean) ** 2)
c_mse = (c_mse_1 + c_mse_2) / 3
#  3. Cumulative sum
c_mse += c_mse << 1
c_mse += c_mse << 2  # element 0 contains the result
print("\n5. Rescaling & Mod Switching.")
print("->\tMean: ", c_mean)
print("->\tMSE_1: ", c_mse_1)
print("->\tMSE_2: ", c_mse_2)
print("->\tMSE: ", c_mse)

In [ ]:
r_x = HE.decryptFrac(ctxt_x)
r_y = HE.decryptFrac(ctxt_y)
rccSum = HE.decryptFrac(ccSum)
rccSub = HE.decryptFrac(ccSub)
rccMul = HE.decryptFrac(ccMul)
rcSq = HE.decryptFrac(cSq)
rcNeg = HE.decryptFrac(cNeg)
rcRotR = HE.decryptFrac(cRotR)
rcRotL = HE.decryptFrac(cRotL)
rcpSum = HE.decryptFrac(cpSum)
rcpSub = HE.decryptFrac(cpSub)
rcpMul = HE.decryptFrac(cpMul)
rmean = HE.decryptFrac(c_mean)
rmse = HE.decryptFrac(c_mse)

# Note: results are approximate! if you increase the decimals, you will notice
#  the errors
_r = lambda x: np.round(x, decimals=3)
print("6. Decrypting results")
print(" Original ciphertexts: ")
print("   ->\tctxt_x --(decr)--> ", _r(r_x))
print("   ->\tctxt_y --(decr)--> ", _r(r_y))
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt_x + ctxt_y = ccSum --(decr)--> ", _r(rccSum))
print("   ->\tctxt_x - ctxt_y = ccSub --(decr)--> ", _r(rccSub))
print("   ->\tctxt_x * ctxt_y = ccMul --(decr)--> ", _r(rccMul))
print(" Single ciphertext: ")
print("   ->\tctxt_x**2      = cSq   --(decr)--> ", _r(rcSq))
print("   ->\t- ctxt_x       = cNeg  --(decr)--> ", _r(rcNeg))
print("   ->\tctxt_x >> 4    = cRotR --(decr)--> ", _r(rcRotR))
print("   ->\tctxt_x << 4    = cRotL --(decr)--> ", _r(rcRotL))
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt_x + ptxt_y = cpSum --(decr)--> ", _r(rcpSum))
print("   ->\tctxt_x - ptxt_y = cpSub --(decr)--> ", _r(rcpSub))
print("   ->\tctxt_x * ptxt_y = cpMul --(decr)--> ", _r(rcpMul))
print(" Mean Squared error: ")
print("   ->\tmean(ctxt_x, ctxt_y) = c_mean --(decr)--> ", _r(rmean))
print("   ->\tmse(ctxt_x, ctxt_y)  = c_mse  --(decr)--> ", _r(rmse))

In [ ]:
from olm import Account, OutboundGroupSession, InboundGroupSession, OlmGroupSessionError

In [ ]:
from olm import Account, OutboundSession, InboundSession

# Create Alice's account
alice = Account()

# Create Bob's account
bob = Account()

# Generate one-time keys for Bob's account
bob.generate_one_time_keys(1)

# Get Bob's identity key and one-time key
bob_identity_key = bob.identity_keys["curve25519"]
bob_one_time_key = list(bob.one_time_keys["curve25519"].values())[0]

# Create an outbound session for Alice
alice_outbound = OutboundSession(alice, bob_identity_key, bob_one_time_key)

# Encrypt the initial message using Alice's outbound session
initial_plaintext = "Hello, Bob!"
initial_ciphertext = alice_outbound.encrypt(initial_plaintext)

# Create an inbound session for Bob using the initial ciphertext
bob_inbound = InboundSession(bob, initial_ciphertext)

# Decrypt the initial message using Bob's inbound session
decrypted_initial_plaintext = bob_inbound.decrypt(initial_ciphertext)
print("Decrypted initial message:", decrypted_initial_plaintext)

# Encrypt a response from Bob to Alice
response_plaintext = "Hello, Alice!"
response_ciphertext = bob_inbound.encrypt(response_plaintext)

# Decrypt the response using Alice's outbound session
decrypted_response = alice_outbound.decrypt(response_ciphertext)
print("Decrypted response:", decrypted_response)

In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Generate a Diffie-Hellman parameter set
dh_parameters = dh.generate_parameters(
    generator=2, key_size=2048, backend=default_backend()
)

# Alice's private and public keys
alice_private_key = dh_parameters.generate_private_key()
alice_public_key = alice_private_key.public_key()

# Bob's private and public keys
bob_private_key = dh_parameters.generate_private_key()
bob_public_key = bob_private_key.public_key()

# Alice sends her public key to Bob
bob_received_alice_public_key = alice_public_key

# Bob sends his public key to Alice
alice_received_bob_public_key = bob_public_key

# Alice computes the shared secret key
alice_shared_secret = alice_private_key.exchange(bob_received_alice_public_key)

# Bob computes the shared secret key
bob_shared_secret = bob_private_key.exchange(alice_received_bob_public_key)

# Derive a symmetric key from the shared secret key
alice_symmetric_key = alice_shared_secret[:16]  # 128-bit AES key
bob_symmetric_key = bob_shared_secret[:16]  # 128-bit AES key

# Encrypt data using AES
cipher = Cipher(
    algorithms.AES(alice_symmetric_key),
    modes.GCM(b"\x00" * 12),
    backend=default_backend(),
)
encryptor = cipher.encryptor()
ct = encryptor.update(b"Hello, World!") + encryptor.finalize()
tag = encryptor.tag

# Decrypt data using AES
cipher = Cipher(
    algorithms.AES(bob_symmetric_key),
    modes.GCM(b"\x00" * 12, tag),
    backend=default_backend(),
)
decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()

print("Decrypted plaintext:", pt.decode())

In [ ]:
import socket
import threading
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def generate_diffie_hellman_parameters():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    return parameters


def generate_diffie_hellman_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key


def derive_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
    ).derive(shared_key)
    return derived_key


def encrypt_message(key, message):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    padded_message = message + b" " * (16 - len(message) % 16)
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()
    return ciphertext


def decrypt_message(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_message = decryptor.update(ciphertext) + decryptor.finalize()
    message = padded_message.rstrip(b" ")
    return message


def handle_client(client_socket, address):
    print(f"Connected to client: {address}")

    # Generate Diffie-Hellman parameters and keys
    parameters = generate_diffie_hellman_parameters()
    server_private_key, server_public_key = generate_diffie_hellman_keys(parameters)

    # Send server's public key to the client
    client_socket.send(
        server_public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo,
        )
    )

    # Receive client's public key
    client_public_key_pem = client_socket.recv(2048)
    client_public_key = serialization.load_pem_public_key(client_public_key_pem)

    # Derive shared key using Diffie-Hellman
    shared_key = derive_key(server_private_key, client_public_key)

    while True:
        # Receive encrypted message from the client
        encrypted_message = client_socket.recv(1024)
        if not encrypted_message:
            break

        # Decrypt the message using the shared key
        decrypted_message = decrypt_message(shared_key, encrypted_message)
        print(f"Received message from {address}: {decrypted_message.decode()}")

        # Encrypt a response using the shared key and send it back to the client
        response = "Server received your message.".encode()
        encrypted_response = encrypt_message(shared_key, response)
        client_socket.send(encrypted_response)

    client_socket.close()


def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(("localhost", 12345))
    server_socket.listen(1)
    print("Server is listening on localhost:12345")

    while True:
        client_socket, address = server_socket.accept()
        client_thread = threading.Thread(
            target=handle_client, args=(client_socket, address)
        )
        client_thread.start()


def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(("localhost", 12345))
    print("Connected to server")

    # Receive server's public key
    server_public_key_pem = client_socket.recv(2048)
    server_public_key = serialization.load_pem_public_key(server_public_key_pem)

    # Generate client's Diffie-Hellman keys
    parameters = dh.DHParameterNumbers(
        server_public_key.public_numbers().parameter_numbers.p,
        server_public_key.public_numbers().parameter_numbers.g,
    ).parameters()
    client_private_key, client_public_key = generate_diffie_hellman_keys(parameters)

    # Send client's public key to the server
    client_socket.send(
        client_public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo,
        )
    )

    # Derive shared key using Diffie-Hellman
    shared_key = derive_key(client_private_key, server_public_key)

    while True:
        # Get user input and encrypt the message using the shared key
        message = input("Enter a message (or press Enter to quit): ")
        if not message:
            break
        encrypted_message = encrypt_message(shared_key, message.encode())
        client_socket.send(encrypted_message)

        # Receive encrypted response from the server and decrypt it
        encrypted_response = client_socket.recv(1024)
        decrypted_response = decrypt_message(shared_key, encrypted_response)
        print(f"Server response: {decrypted_response.decode()}")

    client_socket.close()


if __name__ == "__main__":
    while True:
        choice = input(
            'Enter "s" to start the server, "c" to start a client, or "q" to quit: '
        )
        if choice == "s":
            start_server()
        elif choice == "c":
            start_client()
        elif choice == "q":
            break
        else:
            print("Invalid choice. Please try again.")

In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def generate_diffie_hellman_parameters():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    return parameters


def generate_diffie_hellman_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key


def derive_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
    ).derive(shared_key)
    return derived_key


def encrypt_message(key, message):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    padded_message = message + b" " * (16 - len(message) % 16)
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()
    return ciphertext


def decrypt_message(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_message = decryptor.update(ciphertext) + decryptor.finalize()
    message = padded_message.rstrip(b" ")
    return message


def simulate_communication():
    # Generate Diffie-Hellman parameters
    parameters = generate_diffie_hellman_parameters()

    # Generate Alice's Diffie-Hellman keys
    alice_private_key, alice_public_key = generate_diffie_hellman_keys(parameters)

    # Generate Bob's Diffie-Hellman keys
    bob_private_key, bob_public_key = generate_diffie_hellman_keys(parameters)

    # Alice derives the shared key
    alice_shared_key = derive_key(alice_private_key, bob_public_key)

    # Bob derives the shared key
    bob_shared_key = derive_key(bob_private_key, alice_public_key)

    # Alice encrypts a message using the shared key
    alice_message = "Hello, Bob!".encode()
    alice_encrypted_message = encrypt_message(alice_shared_key, alice_message)

    # Bob decrypts Alice's message using the shared key
    bob_decrypted_message = decrypt_message(bob_shared_key, alice_encrypted_message)
    print("Bob received:", bob_decrypted_message.decode())

    # Bob encrypts a response using the shared key
    bob_response = "Hi, Alice!".encode()
    bob_encrypted_response = encrypt_message(bob_shared_key, bob_response)

    # Alice decrypts Bob's response using the shared key
    alice_decrypted_response = decrypt_message(alice_shared_key, bob_encrypted_response)
    print("Alice received:", alice_decrypted_response.decode())


if __name__ == "__main__":
    simulate_communication()

In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def generate_diffie_hellman_parameters():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    return parameters


def generate_diffie_hellman_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key


def derive_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
    ).derive(shared_key)
    return derived_key


def encrypt_message(key, message):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    padded_message = message + b" " * (16 - len(message) % 16)
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()
    return ciphertext


def decrypt_message(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_message = decryptor.update(ciphertext) + decryptor.finalize()
    message = padded_message.rstrip(b" ")
    return message


def simulate_communication(num_clients):
    # Generate Diffie-Hellman parameters
    parameters = generate_diffie_hellman_parameters()

    # Generate server's Diffie-Hellman keys
    server_private_key, server_public_key = generate_diffie_hellman_keys(parameters)

    # Generate clients' Diffie-Hellman keys
    client_keys = [generate_diffie_hellman_keys(parameters) for _ in range(num_clients)]

    # Derive shared keys between the server and each client
    shared_keys = [
        derive_key(server_private_key, client_public_key)
        for _, client_public_key in client_keys
    ]

    # Simulate communication between the server and clients
    for i in range(num_clients):
        print(f"Communication with Client {i+1}:")

        # Server encrypts a message using the shared key
        server_message = f"Hello, Client {i+1}!".encode()
        server_encrypted_message = encrypt_message(shared_keys[i], server_message)

        # Client decrypts the server's message using the shared key
        client_private_key = client_keys[i][0]
        client_shared_key = derive_key(client_private_key, server_public_key)
        client_decrypted_message = decrypt_message(
            client_shared_key, server_encrypted_message
        )
        print(f"Client {i+1} received:", client_decrypted_message.decode())

        # Client encrypts a response using the shared key
        client_response = f"Hi, Server! This is Client {i+1}.".encode()
        client_encrypted_response = encrypt_message(client_shared_key, client_response)

        # Server decrypts the client's response using the shared key
        server_decrypted_response = decrypt_message(
            shared_keys[i], client_encrypted_response
        )
        print("Server received:", server_decrypted_response.decode())

        print()


if __name__ == "__main__":
    num_clients = int(input("Enter the number of clients: "))
    simulate_communication(num_clients)

In [ ]:
import Pyfhel

# Data encoding and encryption
encoded_data = encode_data(input_data)
encrypted_data = pyfhel.encrypt(encoded_data)

# Initialization
num_clusters = k
initial_centroids = initialize_centroids(k, encoded_data)
encrypted_centroids = pyfhel.encrypt(initial_centroids)

# Iterative clustering
converged = False
while not converged:
    cluster_assignments = []
    new_centroids = [[] for _ in range(num_clusters)]

    # Assign data points to clusters
    for data_point in encrypted_data:
        distances = [
            pyfhel.distance(data_point, centroid) for centroid in encrypted_centroids
        ]
        cluster_idx = pyfhel.argmin(distances)
        cluster_assignments.append(cluster_idx)
        new_centroids[cluster_idx].append(data_point)

    # Update centroids
    new_encrypted_centroids = []
    for cluster in new_centroids:
        if cluster:
            centroid = pyfhel.mean(cluster)
            new_encrypted_centroids.append(centroid)
        else:
            new_encrypted_centroids.append(encrypted_centroids[cluster_idx])

    # Check for convergence
    converged = check_convergence(encrypted_centroids, new_encrypted_centroids)
    encrypted_centroids = new_encrypted_centroids

# Decryption and result interpretation
decrypted_centroids = [pyfhel.decrypt(centroid) for centroid in encrypted_centroids]
decrypted_assignments = [
    pyfhel.decrypt(assignment) for assignment in cluster_assignments
]
print_clustering_results(decrypted_assignments, decrypted_centroids)

In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


clients = [1 ,2, 3]
num_clients = len(clients)

In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle

clients = [1, 2, 3]
num_clients = len(clients)

def generate_diffie_hellman_parameters():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    return parameters


def generate_diffie_hellman_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key


def derive_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
    ).derive(shared_key)
    return derived_key


def encrypt_message(key, message):
    serialized_obj = pickle.dumps(message)
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    padded_obj = serialized_obj + b" " * (16 - len(serialized_obj) % 16)
    ciphertext = encryptor.update(padded_obj) + encryptor.finalize()
    return ciphertext


def decrypt_message(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_obj = decryptor.update(ciphertext) + decryptor.finalize()
    serialized_obj = padded_obj.rstrip(b" ")
    obj = pickle.loads(serialized_obj)
    return obj

parameters = generate_diffie_hellman_parameters()

server_private_key, server_public_key = generate_diffie_hellman_keys(parameters)
client_keys = [generate_diffie_hellman_keys(parameters) for _ in range(num_clients)]
shared_keys = [
    derive_key(server_private_key, client_public_key)
    for _, client_public_key in client_keys
]

server_message = f"Hello, Clientjkhgjkh {1}!"
server_encrypted_message = encrypt_message(shared_keys[0], server_message)
print(server_encrypted_message)

In [ ]:
parameters = generate_diffie_hellman_parameters()

server_private_key, server_public_key = generate_diffie_hellman_keys(parameters)
client_keys = [generate_diffie_hellman_keys(parameters) for _ in range(num_clients)]
shared_keys = [
    derive_key(server_private_key, client_public_key)
    for _, client_public_key in client_keys
]

server_message = f"Hello, Clientjkhgjkh {1}!"
server_encrypted_message = encrypt_message(shared_keys[0], server_message)
print(server_encrypted_message)

In [ ]:
client_private_key = client_keys[0][0]
client_shared_key = derive_key(client_private_key, server_public_key)
client_decrypted_message = decrypt_message(
    client_shared_key, server_encrypted_message
)
print(f"Client {0} received:", client_decrypted_message)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
epochs = 5
clients = [1, 2, 3]
num_clients = len(clients)

# Assuming you have defined your model architecture
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc1(x)


# Dummy encryption function (replace with actual implementation)
def encrypt_wt(weights, client_id):
    return weights


# Dummy decryption function (replace with actual implementation)
def decrypt_weights(encrypted_weights):
    return encrypted_weights


# Dummy aggregation function (replace with actual implementation)
def aggregate_wt(cwts):
    aggregated_weights = cwts[0]  # Simplified for demonstration
    for weights in cwts[1:]:
        for i in range(len(weights)):
            aggregated_weights[i] += weights[i]
    for i in range(len(aggregated_weights)):
        aggregated_weights[i] /= len(cwts)
    return aggregated_weights


# Function to train the model
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    accuracy = evaluate_model(model, train_loader)
    return model, accuracy


# Function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = correct / total
    return accuracy


# Initialize models, datasets, and other variables
models = [SimpleModel() for _ in range(num_clients)]
datasets = [
    DataLoader(dataset, batch_size=32, shuffle=True) for dataset in dataset_parts
]
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=0.01) for model in models]

# Encrypt initial weights
cwts = [
    encrypt_wt([param.data.clone() for param in model.parameters()], i)
    for i, model in enumerate(models)
]

for e in tqdm(range(epochs)):
    # Aggregate weights
    cwts = aggregate_wt(cwts)
    # Decrypt aggregated weights
    wts = decrypt_weights(cwts)
    # Clear the encrypted weights list
    cwts = []
    # Train each model with its respective dataset
    accuracies = []
    for wt, model, dataset, optimizer, i in zip(
        wts, models, datasets, optimizers, range(len(models))
    ):
        # Set model weights
        for param, new_param in zip(model.parameters(), wt):
            param.data.copy_(new_param)

        # Train the model
        model, accuracy = train_model(model, dataset, criterion, optimizer)
        accuracies.append(accuracy)
        print("accuracies", accuracy)

        # Get and encrypt updated weights
        wtarray = [param.data.clone() for param in model.parameters()]
        cwts.append(encrypt_wt(wtarray, i))

    print(f"Epoch {e+1}/{epochs}, Accuracies: {accuracies}")

NameError: name 'dataset_parts' is not defined

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm

# Number of clients
num_clients = 3
epochs = 10


# Sample Dataset
class SampleDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.data = torch.randn(size, 10)  # 10 features
        self.labels = torch.randint(0, 5, (size,))  # 5 classes

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Generate synthetic dataset
full_dataset = SampleDataset(size=3000)
dataset_parts = random_split(full_dataset, [1000, 1000, 1000])
dataloaders = [DataLoader(part, batch_size=32, shuffle=True) for part in dataset_parts]


# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc1(x)


# Dummy encryption function (replace with actual implementation)
def encrypt_wt(weights, client_id):
    return [w.clone() for w in weights]


# Dummy decryption function (replace with actual implementation)
def decrypt_weights(encrypted_weights):
    return [w.clone() for w in encrypted_weights]


# Corrected aggregation function
def aggregate_wt(cwts):
    num_clients = len(cwts)
    aggregated_weights = [torch.zeros_like(w) for w in cwts[0]]

    for weights in cwts:
        for i, w in enumerate(weights):
            aggregated_weights[i] += w

    for i in range(len(aggregated_weights)):
        aggregated_weights[i] /= num_clients

    return aggregated_weights


# Function to train the model
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    accuracy = evaluate_model(model, train_loader)
    return model, accuracy


# Function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = correct / total
    return accuracy


# Initialize models, datasets, and other variables
models = [SimpleModel() for _ in range(num_clients)]
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=0.01) for model in models]

# Encrypt initial weights
cwts = [
    encrypt_wt([param.data.clone() for param in model.parameters()], i)
    for i, model in enumerate(models)
]

for e in tqdm(range(epochs)):
    # Aggregate weights
    cwts = aggregate_wt(cwts)
    # Decrypt aggregated weights
    wts = decrypt_weights(cwts)
    # Clear the encrypted weights list
    cwts = []
    # Train each model with its respective dataset
    accuracies = []
    for wt, model, dataloader, optimizer, i in zip(
        wts, models, dataloaders, optimizers, range(len(models))
    ):
        # Debug: Print shapes of weights and parameters
        for param, new_param in zip(model.parameters(), wt):
            print(f"param shape: {param.shape}, new_param shape: {new_param.shape}")

        # Set model weights
        for param, new_param in zip(model.parameters(), wt):
            if param.shape == new_param.shape:
                param.data.copy_(new_param)
            else:
                raise ValueError(
                    f"Shape mismatch: param shape {param.shape}, new_param shape {new_param.shape}"
                )

        # Train the model
        model, accuracy = train_model(model, dataloader, criterion, optimizer)
        accuracies.append(accuracy)
        print("accuracies", accuracy)

        # Get and encrypt updated weights
        wtarray = [param.data.clone() for param in model.parameters()]
        cwts.append(encrypt_wt(wtarray, i))

    print(f"Epoch {e+1}/{epochs}, Accuracies: {accuracies}")

  0%|          | 0/10 [00:00<?, ?it/s]

param shape: torch.Size([5, 10]), new_param shape: torch.Size([10])
param shape: torch.Size([5]), new_param shape: torch.Size([10])


ValueError: Shape mismatch: param shape torch.Size([5, 10]), new_param shape torch.Size([10])

In [1]:
import torch
import torch.nn as nn
import numpy as np


# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize the model
model = SimpleModel()

# Get model weights as a list of NumPy arrays
weights = [param.data.numpy() for param in model.parameters()]

# Print the weights
# for i, weight in enumerate(weights):
#     print(f"Weight {i} shape: {weight.shape}")
#     print(weight)
weights[0]

array([[ 0.23092234, -0.24072152,  0.05027333,  0.160478  , -0.03642218,
        -0.20114903,  0.2855035 , -0.1988375 ,  0.06521879,  0.24452202],
       [ 0.22331414, -0.25304052, -0.03493283, -0.15063168, -0.16471407,
        -0.12465105, -0.28285986,  0.28227222,  0.25317663, -0.24842949],
       [-0.15352544,  0.12528935, -0.23829462,  0.14931229, -0.07612436,
         0.2910646 , -0.26801547, -0.2275473 , -0.3016971 ,  0.28809458],
       [ 0.19160898, -0.02669213, -0.02650805, -0.02757097,  0.06617284,
         0.31430525,  0.24564186,  0.09348659,  0.09282531,  0.261958  ],
       [ 0.03892452, -0.14820458,  0.1074753 , -0.21575932,  0.03476233,
         0.07165267, -0.2646952 , -0.1352763 ,  0.22311859, -0.22385524]],
      dtype=float32)

In [2]:
def get_weights(model):
    return [param.cpu().detach().numpy() for param in model.parameters()]

In [3]:
print(wtarray := get_weights(model))

[array([[ 0.23092234, -0.24072152,  0.05027333,  0.160478  , -0.03642218,
        -0.20114903,  0.2855035 , -0.1988375 ,  0.06521879,  0.24452202],
       [ 0.22331414, -0.25304052, -0.03493283, -0.15063168, -0.16471407,
        -0.12465105, -0.28285986,  0.28227222,  0.25317663, -0.24842949],
       [-0.15352544,  0.12528935, -0.23829462,  0.14931229, -0.07612436,
         0.2910646 , -0.26801547, -0.2275473 , -0.3016971 ,  0.28809458],
       [ 0.19160898, -0.02669213, -0.02650805, -0.02757097,  0.06617284,
         0.31430525,  0.24564186,  0.09348659,  0.09282531,  0.261958  ],
       [ 0.03892452, -0.14820458,  0.1074753 , -0.21575932,  0.03476233,
         0.07165267, -0.2646952 , -0.1352763 ,  0.22311859, -0.22385524]],
      dtype=float32), array([-0.12221324,  0.18987517, -0.30763707,  0.19206138,  0.30669737],
      dtype=float32), array([[ 0.18580586, -0.39679396,  0.25478312, -0.37008   ,  0.03028554],
       [ 0.02669882,  0.1924318 , -0.06256127, -0.22779536,  0.27843192]

In [4]:
def load_weights(model, weights):
    with torch.no_grad():
        for param, weight in zip(model.parameters(), weights):
            param.copy_(torch.tensor(weight))
    return model

In [5]:
print(model := load_weights(model, wtarray))

SimpleModel(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)


In [9]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()
ckks_params = {
    "scheme": "CKKS",
    "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
    "scale": 2**30,  # All the encodings will use it for float->fixed point
    "qi_sizes": [60, 30, 30, 30, 60],  # Number of bits of each prime in the chain.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()

In [10]:
def encrypt_wt(wtarray, i):
    cwt = []
    for layer in wtarray:
        flat_array = layer.astype(np.float64).flatten()

        chunks = np.array_split(flat_array, (len(flat_array) + 2**10 - 1) // 2**10)
        clayer = []
        for chunk in chunks:
            ptxt = HE.encodeFrac(chunk)
            ctxt = HE.encryptPtxt(ptxt)
            clayer.append(ctxt)
        cwt.append(clayer.copy())
    # ciphertext = encrypt_message_AES(client_shared_keys[i], cwt)
    
    return cwt

In [21]:
cwt = encrypt_wt(wtarray, 1)

In [22]:
models = [SimpleModel() for _ in range(4)]

In [23]:
def decrypt_weights(res):
    decrypted_weights = []
    for client_weights, model in zip(res, models):
        decrypted_client_weights = []
        wtarray = get_weights(model)
        for layer_weights, layer in zip(client_weights, wtarray):
            decrypted_layer_weights = []
            flat_array = layer.astype(np.float64).flatten()
            chunks = np.array_split(flat_array, (len(flat_array) + 2**10 - 1) // 2**10)
            for chunk, encrypted_chunk in zip(chunks, layer_weights):
                decrypted_chunk = HE.decryptFrac(encrypted_chunk)
                original_chunk_size = len(chunk)
                decrypted_chunk = decrypted_chunk[:original_chunk_size]
                decrypted_layer_weights.append(decrypted_chunk)
            decrypted_layer_weights = np.concatenate(decrypted_layer_weights, axis=0)
            decrypted_layer_weights = decrypted_layer_weights.reshape(layer.shape)
            decrypted_client_weights.append(decrypted_layer_weights)
        decrypted_weights.append(decrypted_client_weights)
    return decrypted_weights

In [24]:
bwt = [cwt for _ in range(4)]

In [25]:
wt = decrypt_weights(bwt)

In [28]:
load_weights(models[0], wt[0])

SimpleModel(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)